## Import libraries

In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings("ignore")

## Import data

In [5]:
# Link
link = r'E:\datasets\car\all_anonymized_2015_11_2017_03.csv'

# Preview
df = pd.read_csv(link)
df.head()

,maker,model,mileage,manufacture_year,engine_displacement,engine_power,body_type,color_slug,stk_year,transmission,door_count,seat_count,fuel_type,date_created,date_last_seen,price_eur
0,ford,galaxy,151000.0,2011.0,2000.0,103.0,NaN,NaN,None,man,5,7,diesel,2015-11-14 18:10:06.838319+00,2016-01-27 20:40:15.46361+00,10584.75
1,skoda,octavia,143476.0,2012.0,2000.0,81.0,NaN,NaN,None,man,5,5,diesel,2015-11-14 18:10:06.853411+00,2016-01-27 20:40:15.46361+00,8882.31
2,bmw,NaN,97676.0,2010.0,1995.0,85.0,NaN,NaN,None,man,5,5,diesel,2015-11-14 18:10:06.861792+00,2016-01-27 20:40:15.46361+00,12065.06
3,skoda,fabia,111970.0,2004.0,1200.0,47.0,NaN,NaN,None,man,5,5,gasoline,2015-11-14 18:10:06.872313+00,2016-01-27 20:40:15.46361+00,2960.77
4,skoda,fabia,128886.0,2004.0,1200.0,47.0,NaN,NaN,None,man,5,5,gasoline,2015-11-14 18:10:06.880335+00,2016-01-27 20:40:15.46361+00,2738.71


## Cleaning

### model

In [6]:
# Count of different makers for each model
model_counts = df.groupby('model')['maker'].nunique().sort_values(ascending=False)

# Filter more than 2
model_counts = model_counts[model_counts>1]
model_counts


model
octavia               8
passat                6
200                   6
golf                  5
grand-cherokee        4
                     ..
kangoo                2
range-rover-evoque    2
126                   2
range-rover-sport     2
rapid                 2
Name: maker, Length: 109, dtype: int64

<b> Fix outliers : </b> we check the proportion of the most frequent model. If the model has the maximum maker with frequency over 95%, we update all of the other makers to that one.

In [7]:
# Define function
def model_frequency(df):
    model = df['model'].values[0]
    counts = df['maker'].value_counts()
    perc = counts[0] / counts.sum() * 100
    return pd.DataFrame({'agg1': [counts.index[0]], 'agg2': [perc]})

# Create agg_df as the aggregated dataframe
agg_df = df.groupby('model').apply(lambda x: model_frequency(x))
agg_df = agg_df.reset_index().drop('level_1', axis=1)
agg_df.columns = ['Model', 'Most frequent maker', 'Frequency']

# Filter agg_df (frequency > 0.95). Get dictionary
agg_df = agg_df.loc[agg_df['Frequency'] >= 95, ['Model', 'Most frequent maker']]
dct = agg_df.set_index('Model')['Most frequent maker'].to_dict()

# Update
df.loc[df['model'].isin(dct.keys()), 'maker'] = df.loc[df['model'].isin(dct.keys()),'model'].map(dct)

### manufacture_year

<b> Fix outliers : </b>
1. We filter out values with manufacture year less than 1960
2. We calculate median and std of each maker-model pair
3. We estimate the first_manufacture_year to be (median - std * 1.5) 

In [8]:
cutoff_year = 1990

# Get median and std of each maker-mode pair, and estimate first manufacture year
maker_model_df = df.dropna(subset = ['maker', 'model'])
maker_model_df = maker_model_df[maker_model_df['manufacture_year'] >= cutoff_year]
maker_model_agg = maker_model_df.groupby(['maker', 'model'], as_index=False)['manufacture_year'].agg({'median' : np.median, 'std' : np.std})
maker_model_agg = maker_model_agg.fillna(0)

# Estimate first manufacture year
maker_model_agg['first_manufacture_year'] = (maker_model_agg['median'] - maker_model_agg['std'] * 1.5).astype('int32')
maker_model_agg.head()

,maker,model,median,std,first_manufacture_year
0,alfa-romeo,145,1999.0,4.866797,1991
1,alfa-romeo,146,2000.0,4.051587,1993
2,alfa-romeo,147,2004.0,2.717781,1999
3,alfa-romeo,155,1996.5,2.629956,1992
4,alfa-romeo,156,2002.0,3.272549,1997


In [9]:
# Merged to the orginal data
merged_df = df.merge(maker_model_agg[['maker', 'model', 'median', 'first_manufacture_year']], how='left', on = ['maker', 'model'] )

# Update (fill wrong or missing values with median year)
merged_df.loc[~merged_df['first_manufacture_year'].isna(), 'manufacture_year'] = \
    merged_df.loc[~merged_df['first_manufacture_year'].isna(), ['manufacture_year', 'first_manufacture_year', 'median']] \
    .apply(lambda x: x['median'] if x['manufacture_year'] <= x['first_manufacture_year']  else x['manufacture_year'], axis=1)
df = merged_df.drop(['first_manufacture_year', 'median'], axis=1)

df.loc[df['manufacture_year'] < cutoff_year ,'manufacture_year'] = np.nan

### mileage

In [10]:
# Cutoff mileage:
cutoff_mileage = 400000

# Update
df.loc[df['mileage'] > cutoff_mileage ,'mileage'] = np.nan

### stk_year

In [11]:
# Replace None with np.nan and change the column type to float
df.loc[df['stk_year'] == 'None', 'stk_year'] = np.nan
df['stk_year'] = df['stk_year'].astype(float)

# Update
df.loc[df['stk_year'] > 2023.0, 'stk_year'] = np.nan
df.loc[df['stk_year'] > 2020.0, 'stk_year'] = 2020.0

### body_type

In [12]:
# Change 'other' type to nan
df.loc[df['body_type'] == 'other', 'body_type'] = np.nan

# Change 2020.0 to string 2020
df.loc[df['body_type'] == 2020.0, 'body_type'] = '2020'

In [13]:
# Create dataframe of maker-model pair and the most frequent body_type
def mode(ser):
    return ser.dropna().mode().iloc[0] if not ser.mode().empty else np.nan

temp_df = df[["maker", "model",'body_type']].dropna(subset = ["maker", "model"] , how='all')
agg_df = temp_df.groupby(['maker', 'model'], as_index=False, dropna=False)['body_type'].apply(mode)
agg_df = agg_df.rename({'body_type' : 'new_body_type'}, axis=1)

# Merge
merged_df = df.merge(agg_df, how='left', on=["maker", "model"])

# Update
merged_df.loc[merged_df['body_type'].isna(), 'body_type'] = merged_df.loc[merged_df['body_type'].isna(), 'new_body_type']
df = merged_df.drop('new_body_type', axis=1)


### engine_displacement

In [14]:
# 8500>  >650
df.loc[(df['engine_displacement'] > 8500) | (df['engine_displacement'] < 600), 'engine_displacement'] = np.nan

In [15]:
# Create dataframe of maker-model pair and the median engine_displacement
temp_df = df[["maker", "model",'engine_displacement']].dropna(subset = ["maker", "model"] , how='all')
agg_df = temp_df.groupby(['maker', 'model'], as_index=False, dropna=False)['engine_displacement'].median()
agg_df = agg_df.rename({'engine_displacement' : 'new_engine_displacement'}, axis=1)

# Merge
merged_df = df.merge(agg_df, how='left', on=["maker", "model"])

# Update
merged_df.loc[merged_df['engine_displacement'].isna(), 'engine_displacement'] = merged_df.loc[merged_df['engine_displacement'].isna(), 'new_engine_displacement']
df = merged_df.drop('new_engine_displacement', axis=1)

### engine_power

In [16]:
# 500>  >65
df["engine_power"] = df["engine_power"] * 1.34102
df.loc[(df['engine_power'] > 500) | (df['engine_power'] < 65), 'engine_power'] = np.nan

In [17]:
# Create dataframe of maker-model pair and the median engine_power
temp_df = df[["maker", "model", 'transmission','engine_power']].dropna(subset = ["maker", "model", 'transmission'] , how='all')
agg_df = temp_df.groupby(['maker', 'model', 'transmission'], as_index=False, dropna=False)['engine_power'].median()
agg_df = agg_df.rename({'engine_power' : 'new_engine_power'}, axis=1)

# Merge
merged_df = df.merge(agg_df, how='left', on=["maker", "model", 'transmission'])

# Update
merged_df.loc[merged_df['engine_power'].isna(), 'engine_power'] = merged_df.loc[merged_df['engine_power'].isna(), 'new_engine_power']
df = merged_df.drop('new_engine_power', axis=1)

### Last: price_eur

In [18]:
# Update
df = df[df['price_eur'] != 1295.34]

cutoff_price = 100000
df = df[df['price_eur'] <= cutoff_price]

In [54]:
### Bonus
StatusFrame = df.notnull()

Occurences = StatusFrame.value_counts().reset_index()
Occurences

,maker,model,mileage,manufacture_year,engine_displacement,engine_power,body_type,color_slug,stk_year,transmission,door_count,seat_count,fuel_type,date_created,date_last_seen,price_eur,0
0,True,True,True,True,True,True,True,False,False,True,True,True,False,True,True,True,535267
1,True,True,True,True,True,True,True,False,False,True,True,True,True,True,True,True,477636
2,True,True,True,True,True,True,True,False,True,True,True,True,False,True,True,True,217729
3,False,False,True,True,True,True,False,False,False,True,True,True,True,True,True,True,201584
4,True,False,True,True,True,True,True,False,False,True,True,True,True,True,True,True,198697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,False,False,False,True,True,True,False,False,True,False,True,True,False,True,True,True,1
569,True,True,False,False,True,False,False,False,False,False,False,False,False,True,True,True,1
570,True,True,True,False,True,False,True,True,False,True,True,True,True,True,True,True,1
571,True,True,True,False,True,False,True,False,True,True,True,False,False,True,True,True,1
